In [12]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("/home/ubuntu/graphseq-inference/")
from graphseq_inference.data_utils import *
from graphseq_inference.models import *
from graphseq_inference.train_utils import *

#!python /home/ubuntu/graphseq-inference/graphseq_inference/generate_alpha.py > /dev/null 2>&1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
def get_mid_trees(ts):

    for i, tree in enumerate(ts.trees()):
        if tree.interval.left >= (ts.sequence_length/2):
            break

    first_tree = i-249
    last_tree = i + 250
    
    trees = []
    for j, tree in enumerate(ts.aslist()):
        if j >= first_tree and j <= last_tree:
            trees.append(tree)
            
    return trees

In [ ]:
directory = "./classification_dataset/"

files = os.listdir(directory)
files = [directory +  file for file in files]

In [ ]:
len(files)

In [16]:
label_lookup = {
    "kingman_selection_none" : 0,
    "kingman_selection_weak" : 1,
    "kingman_selection_medium" : 2,
    "kingman_selection_strong" : 3,
    
    "beta_selection_a1.99-1.75" : 4,
    "beta_selection_a1.75-1.50" : 5,
    "beta_selection_a1.50-1.25" : 6,
    "beta_selection_a1.25-1.01" : 7,    
}

In [ ]:
def preorder_dist(tree):
    result = []
    for root in tree.roots:
        stack = [(root, (-1, root) , (tree.get_time(tree.root), 0))]
        while len(stack) > 0:
            u, pc, time = stack.pop()
            result.append((u, pc, time))
            for v in tree.children(u):
                stack.append((v, (u,v) , (tree.get_time(u),tree.get_time(v))))
    return result

def restructure_result(result):


    is_modified = True
    while True:

        if not is_modified: break
        is_modified = False

        for i, (idx_node, (parent, node), (time_parent, time_node)) in enumerate(result):
            branch_length = time_parent - time_node
            
            
            
            #if branch_length < (time_parent*threshold) and branch_length != 0 and (parent != node):
            

            threshold = 20
            
            if time_parent < 200:
                threshold = 20
            
            if time_parent < 20:
                threshold = 2
                
            if time_parent < 10:
                threshold = 1
                
            if time_parent < 5:
                threshold = 0.5
                
            
            if branch_length < threshold and branch_length != 0 and (parent != node):
    
                
                new_time = (time_parent+time_node)/2
                result[i] = (-1, (parent, parent), (time_parent, time_parent))
                for j, (_, (p, n), (tp, tn)) in enumerate(result):
                    if node == n:
                        result[j] = (-1, (p, parent), (tp, time_parent))#
                    if node == p:
                        result[j] = (-1, (parent, n), (time_parent, tn))

                is_modified = True
                break

    new_result = []
    for a, (b,c), (d, e) in result:
        if b != c:
            new_result.append((a, (b,c), (d,e)))
    return new_result

def multiple_mergerized_to_data_object(result):

    G = nx.Graph()
    for _, edge, times in result[:-1]:
        a, b = edge
        ta, tb = times
        branch = ta - tb
        G.add_weighted_edges_from([(a, b, branch)])
        
    data = from_networkx(G)
    rename_data_attribute(data, "weight", "edge_weight") 

    return data



def tree_to_data_objects(tree, num_embedding=60):
    max_num_nodes = 2 * 10 - 1 

    data = from_networkx(nx.Graph(tree.as_dict_of_dicts()))
    rename_data_attribute(data, "branch_length", "edge_weight") 
    num_nodes = data.num_nodes
    data.x = torch.eye(max_num_nodes,num_embedding)
    data.x[num_nodes:] = torch.zeros(num_embedding)
    data.num_nodes = max_num_nodes
    return data


def tree_to_restructured_data_objects(tree, num_embedding=60):
    result = preorder_dist(tree)
    result.reverse()
    result = restructure_result(result)    
    data = multiple_mergerized_to_data_object(result)
    
    max_num_nodes = 2 * 10 - 1 
    num_nodes = data.num_nodes
    data.x = torch.eye(max_num_nodes,num_embedding)
    data.x[num_nodes:] = torch.zeros(num_embedding)
    data.num_nodes = max_num_nodes
    
    return data

In [ ]:

def get_y_label_from_y_dict(y_dict):
    reproduction_model, parameter = y_dict["model"]

    if reproduction_model == "beta":
        if parameter < 1.25:
            return torch.Tensor([7]).long()
        if parameter < 1.5:
            return torch.Tensor([6]).long()
        if parameter < 1.75:
            return torch.Tensor([5]).long()
        if parameter < 2:
            return torch.Tensor([4]).long()

    else:
        label = y_dict["model"][0] + "_selection_" + y_dict["selection_type"]
        y = torch.Tensor([label_lookup[label]]).long()
        return y

def convert_for_classification(file: tskit.trees.TreeSequence,
                                num_embedding:int = 60, restructure=False,
                           ):
    
    ts, y_dict = torch.load(file)
    
    y = get_y_label_from_y_dict(y_dict)
    
    #label = y_dict["model"][0] + "_selection_" + y_dict["selection_type"]
    #y = torch.Tensor([label_lookup[label]]).long()
    trees = get_mid_trees(ts)
    
        
    if restructure:
        data_objects = [tree_to_restructured_data_objects(tree) for tree in trees]
    else:
        data_objects = [tree_to_data_objects(tree) for tree in trees]
        
    
    for data in data_objects:
        data.y = y
    
    
    #for i, tree in enumerate(trees):            
    #    data = from_networkx(nx.Graph(tree.as_dict_of_dicts()))
    #    rename_data_attribute(data, "branch_length", "edge_weight") 
    #    num_nodes = data.num_nodes
    #    data.x = torch.eye(max_num_nodes,num_embedding)
    #    data.x[num_nodes:] = torch.zeros(num_embedding)
    #    data.y = y
    #    data.num_nodes = max_num_nodes
    #    data_objects.append(data)
        
    return data_objects

In [18]:
file = files[0]
ts, y_dict = torch.load(file)
y_dict

In [21]:
file = files[2]
ts, y_dict = torch.load(file)
y_dict

{'model': ('beta', 1.37),
 'sequence_length': 409600000,
 'selection_coefficient': 0,
 'selection_type': None}

In [93]:


def get_y_label_from_y_dict(y_dict):
    reproduction_model, parameter = y_dict["model"]

    if reproduction_model == "beta":
        if parameter < 1.25:
            return torch.Tensor([7]).long()
        if parameter < 1.5:
            return torch.Tensor([6]).long()
        if parameter < 1.75:
            return torch.Tensor([5]).long()
        if parameter < 2:
            return torch.Tensor([4]).long()

    else:
        label = y_dict["model"][0] + "_selection_" + y_dict["selection_type"]
        y = torch.Tensor([label_lookup[label]]).long()
        return y
    
    

{'model': ('beta', 1.74), 'sequence_length': 25600000, 'selection_coefficient': 0, 'selection_type': None}


tensor([5])

In [ ]:
class ClassificationModel(nn.Module):
    
    def __init__(self, DemographyNet, num_classes, time_window=60):
        super().__init__()
        self.l1 = nn.Linear(time_window, time_window//2)
        self.l2 = nn.Linear(time_window//2, num_classes)
        self.DemographyNet = DemographyNet
    
    def forward(self, batch):
        x = self.DemographyNet(batch)
        return self.l2(F.relu(self.l1(x)))
    
    


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
demography_net = DiffPoolNet(19, 60, 192, 60)
model = ClassificationModel(demography_net, num_classes=8).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
initialize_weights(model)
criterion = nn.CrossEntropyLoss()

In [ ]:
length = 60
loss_all = []

for epoch in range(2):
    np.random.shuffle(files)
    for i, file in enumerate(tqdm(files)):


        data_objects = convert_for_classification(file, restructure=False)

        optimizer.zero_grad()
        dl = DataLoader(data_objects, batch_size=len(data_objects))

        for batch in dl:
            batch = batch.to(device)
            y_hat = model(batch)
            y_hat = y_hat.mean(0)
            y_true = batch.y[0]

            loss = criterion(y_hat, y_true) 
            loss.backward()
            loss_all.append(loss.item())
            optimizer.step()

            if i != 0 and i % 20000 == 0:
                loss_all = np.mean(loss_all)
                print(f"loss {loss_all}")
                torch.save(model.state_dict(), "./classifcation_model/mmc_diffpool_model_classification_norestruct_intermediate" + str(epoch) + "_" + str(i) + ".pth")
                os.system(f'echo "Epoch: {epoch:03d}, Train Loss: {np.mean(loss_all):.4f}" >> ./classifcation_model/mmc_diffpool_model_classification_norestruct_intermediate.txt')
                loss_all = []
                
    torch.save(model.state_dict(), "./classifcation_model/mmc_diffpool_model_classification_norestruct_" + str(epoch) + "_" + str(i) + ".pth")
